In [1]:
import numpy as np
import pandas as pd
import string
import itertools

import warnings
warnings.filterwarnings( "ignore")
import matplotlib.pyplot as plt
%matplotlib inline

from thefuzz import fuzz,process
import participation_utils as plb

from importlib import reload

In [25]:
filename = '../data/Plt_sci_publications_merged_8.09.csv'

data = pd.read_csv(filename)
#data = data.iloc[:,:36]
print(data.shape)

(296447, 53)


In [26]:
data.head()

,Author Full Names,Article Title,Journal,impact factor (2020),Author Keywords,Keywords Plus,Abstract,Addresses--all authors,Nations--all authors,Corresponding author Addresses,...,Corresponding_author_last_name,first and last names,First_names_extracted,First_names_currated,inferred_gender,gender_inference_accuracy,#_samples_gender_inference,Per_capita_income_2020,GDP_2020,_merge
0,"Raho, Nicolas; Fraga, Santiago; Abad, Jose P.;...","Biecheleria tirezensis sp nov (Dinophyceae, Su...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Athalassic; CBC; dinoflagellate; halotolerant;...,INTERNAL TRANSCRIBED SPACER; RIBOSOMAL-RNA; LA...,A new euryhaline and eurythermal dinoflagellat...,"[Raho, Nicolas; Abad, Jose P.; Marin, Irma] Un...",NaN,"Abad, JP; Marin, I (corresponding author), Uni...",...,Abad,"['Abad, Jose P.']",Jose P.,Jose P.,male,100.0,7.0,37756.35100,1.280000e+12,both
1,"Abdel-Hafez, Sobhy I. I.; Abo-Elyousr, Kamal A...",Fungicidal activity of extracellular products ...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Alternaria porri; cyanobacterial extracellular...,BLUE-GREEN; ANTIFUNGAL ACTIVITY; PURPLE BLOTCH...,Although cyanobacteria are recognized as renew...,"[Abdel-Hafez, Sobhy I. I.; Abdel-Rahim, Ismail...",NaN,"Abdel-Rahim, IR (corresponding author), Assiut...",...,Abdel-Rahim,"['Abdel-Rahim, Ismail R.']",Ismail R.,Ismail R.,male,99.0,489.0,NaN,NaN,both
2,"Abdelahad, Nadia; Bolpagni, Rossano; Jona Lasi...","Distribution, morphology and ecological niche ...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Autecology; freshwater red algae; Italy; lowla...,RED ALGAE; HELMINTHOSUM BATRACHOSPERMALES; SEC...,Twenty-three freshwater springs of the Po plai...,"[Abdelahad, Nadia; Amadio, Carolina] Sapienza ...",NaN,"Abdelahad, N (corresponding author), Sapienza ...",...,Abdelahad,"['Abdelahad, Nadia']",Nadia,Nadia,female,98.0,5467.0,41987.81596,1.890000e+12,both
3,"Aboal, Marina; Chapuis, Iara; Paiano, Monica O...","Diversity of Chroothece (Rhodophyta, Stylonema...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Chroothece; cryptic diversity; ecology; phylog...,RED ALGAE RHODOPHYTA; RICHTERIANA RHODOPHYTA; ...,Chroothece has been reported from a range of f...,"[Aboal, Marina] Fac Biol, Dept Plant Biol, Lab...",NaN,"Aboal, M (corresponding author), Fac Biol, Dep...",...,Aboal,"['Aboal, Marina']",Marina,Marina,female,98.0,8143.0,37756.35100,1.280000e+12,both
4,"Aboal, Marina; Eugenia Garcia-Fernandez, Maria...","Ecology, morphology and physiology of Chroothe...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,calcareous Chroothece; colony; lipid; organic ...,DIATOM DIDYMOSPHENIA; ALGAL COMMUNITIES; NITRO...,The ecology of Chroothece was studied in the h...,"[Aboal, Marina; Eugenia Garcia-Fernandez, Mari...",NaN,"Aboal, M (corresponding author), Univ Murcia, ...",...,Aboal,"['Aboal, Marina']",Marina,Marina,female,98.0,8143.0,37756.35100,1.280000e+12,both


In [4]:
for i,c in enumerate(data.columns):
    print(i,c,sep='\t')

0	Author Full Names
1	Article Title
2	Journal
3	impact factor (2020)
4	Author Keywords
5	Keywords Plus
6	Abstract
7	Addresses--all authors
8	Nations--all authors
9	Corresponding author Addresses
10	Corresponding author Nation
11	Corresponding author Continent
12	Corresponding author global designation
13	DOI
14	Email Addresses
15	Researcher Ids
16	ORCIDs
17	Funding Orgs
18	Funding Text
19	Cited Reference Count
20	Times Cited, All Databases
21	Publisher
22	Publisher City
23	Publisher Address
24	ISSN
25	eISSN
26	Open Access Designations
27	Open Access simplified
28	Quartile
29	Journal location
30	Subspecality (JCR)
31	Society (Yes or No)
32	Name of the society
33	Open Access options available
34	Open access fees (USD)
35	Publication Date


In [5]:
idx = 3657
for i,c in enumerate(data.iloc[idx]):
    print(i, data.columns[i], c, sep='\t')

0	Author Full Names	Aberkani, Kamal; Hao, Xiuming; de Halleux, Damien; Dorais, Martine; Vineberg, Stephen; Gosselin, Andre
1	Article Title	Effects of Shading Using a Retractable Liquid Foam Technology on Greenhouse and Plant Microclimates
2	Journal	HORTTECHNOLOGY
3	impact factor (2020)	1.06
4	Author Keywords	heat stress; high temperature; relative humidity; solar radiation; ventilation
5	Keywords Plus	TRANSPIRATION; SUMMER; PHOTOSYNTHESIS; DEFICIT; REGIME
6	Abstract	Climate control is an important aspect of greenhouse crop management. Shading is one popular method for reducing excess solar heat radiation and high air temperatures in the greenhouse during the summer season. A new innovative technology has recently been developed and is based on the injection of liquid foam between the double layers of polyethylene of the greenhouse roof. The foam can be used as a shading method during the warm days of the summer. This is the first investigation into the effect of shading using the liqui

In [28]:
institutes = data['Addresses--all authors']
cinstitutes = data['Corresponding author Addresses']

In [7]:
title = 'Photoinhibition of photosystem I'
title = 'DETECTION AND IDENTIFICATION METHODS AND NEW TESTS AS DEVELOPED AND USED IN THE FRAMEWORK OF COST873 FOR BACTERIA PATHOGENIC TO STONE FRUITS AND NUTS'
data[data['Article Title'] == title].loc[:, ['Author Full Names','DOI']]

,Author Full Names,DOI
255980,"Campillo, T.; Lavire, C.; Shams, M.; Pothier, ...",10.4454/jpp.v94i1sup.016
255981,"Prokic, A.; Gasic, K.; Ivanovic, M. M.; Kuzman...",10.4454/jpp.v94i1sup.020
255982,"Biosca, E. G.; Lopez, M. M.",10.4454/jpp.v94i1sup.018


In [29]:
institutions = dict()
cinstitutions = dict()
articles_ids = dict()
coauth_num = np.zeros(len(data), dtype=np.uint)
corr_num = np.zeros(len(data), dtype=np.uint)

for i in range(len(institutes)):
#for i in range(10):
    institute = institutes[i]
    cinstitute = cinstitutes[i]
    
    institution, cinstitution = plb.get_author_location(institute, cinstitute)
    institutions[i] = institution
    cinstitutions[i] = ','.join(map(str, cinstitution))
    
    articles_ids[i] = np.zeros(len(institution), dtype=np.uint) + i

    coauth_num[i] = len(institution)
    corr_num[i] = len(cinstitution)
    
insts = list(itertools.chain(*institutions.values()))
artcs = list(itertools.chain(*articles_ids.values()))
cinsts = list(itertools.chain(*cinstitutions.values()))

In [30]:
metainfo = pd.DataFrame()

metainfo['coauthor_num'] = coauth_num
metainfo['corresponding_num'] = corr_num
metainfo['corr_institutions'] = cinstitutions.values()

dst = '../results/'
filename = dst + 'author_coauthor_corresponding.csv'
metainfo.to_csv(filename, index=False)

i = 8
rand = np.random.randint(0,len(insts), 25)

for i in rand:
    institute = insts[i]
    _ = split_address(institute, kw=kw, verbose=True)

In [31]:
kw = list(pd.read_csv('../data/inst_keywords.txt', header=None).values.squeeze())
reload(plb)

City = ['' for i in range(len(insts))]
State = ['' for i in range(len(insts))]
Country = ['' for i in range(len(insts))]
ZipCode = ['' for i in range(len(insts))]

for i in range(len(insts)):
    institute = insts[i]
    city, state, country, zip_code = plb.split_address(institute, kw, False)
    
    country = plb.rewrite_country(country)
    
    City[i] = city
    State[i] = state
    Country[i] = country
    ZipCode[i] = zip_code

In [32]:
Address = ['' for i in range(len(insts))]

for i in range(len(insts)):
    if State[i] != '':
        Address[i] = City[i] + ', _' + State[i].replace(' ', '_') + '_, ' + Country[i]
    else:
        Address[i] = City[i] + ', ' + Country[i]

In [33]:
df = pd.DataFrame()
df['original_id'] = artcs
df['full_name'] = insts
df['city'] = City
df['state'] = State
df['country'] = Country
df['zip_code'] = ZipCode
#df['citystate'] = CityState
df['location'] = Address

countries = np.sort(np.unique(Country))

In [34]:
print(len(countries))
countries

201


array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia',
       'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Cook Islands',
       'Costa Rica', 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czechia',
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'England', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Falkland Islands', 'Fiji',
       'Finland', 'France', 'French Guiana', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana', 'Gibraltar'

In [50]:
for loc in df[df['city'] == 'Vic']['full_name']:
    print(loc)

Univ Cent Catalonia UVIC UCC, Univ Vic, CT BETA, Vic, Spain
Cent Univ Catalonia, Univ Vic, Fac Sci & Technol, E-08500 Vic, Spain
Cent Univ Catalonia, Univ Vic, Fac Sci & Technol, E-08500 Vic, Spain


In [51]:
cities = []

for i in range(len(countries)):
    citiesi = set(pd.unique(df.loc[df['country'] == countries[i], 'city']))
    cities.append(citiesi)

In [40]:
#England	Scotland	{'Penicuik', 'Forsinard', 'Thurso'}
#England	Wales	{'Monmouth'}


foo = df[df['city'] == 'Middelburg']
foo[foo['country'] == 'Netherlands']
#foo[foo['country'] == 'Scotland']

,original_id,full_name,city,state,country,zip_code,location
277,102,"Rijksinst Kust & Zee, NL-4330 EA Middelburg, N...",Middelburg,,Netherlands,NL-4330,"Middelburg, Netherlands"
579905,218284,"Koninklijk Zeeuwsch Genootschap Wetenschappen,...",Middelburg,,Netherlands,NL-4330,"Middelburg, Netherlands"


In [52]:
for i in range(len(countries) - 1):
    for j in range(i+1, len(countries)):
        inters = cities[i] & cities[j]
        if len(inters) > 0:
            print(countries[i], countries[j], inters, sep='\t')

Argentina	Bolivia	{'Santa Cruz'}
Argentina	Brazil	{'Formosa', 'Concordia', 'Parana', 'Balcarce', 'Moron', 'Rio Negro', 'San Luis', 'Santa Cruz'}
Argentina	Costa Rica	{'San Pedro'}
Argentina	Ecuador	{'Santa Cruz'}
Argentina	Mexico	{'Cordoba', 'Santa Cruz'}
Argentina	Paraguay	{'San Lorenzo'}
Argentina	Peru	{'San Juan', 'San Martin', 'Ayacucho'}
Argentina	Philippines	{'San Rafael'}
Argentina	Portugal	{'Santa Cruz'}
Argentina	Spain	{'Cordoba', 'La Rioja', 'Posadas'}
Argentina	United States	{'San Carlos', 'Santa Rosa', 'Santa Fe', 'San Francisco', 'San Juan', 'San Rafael', 'Santa Cruz'}
Australia	Bermuda	{'Hamilton'}
Australia	Canada	{'Kingston', 'Hamilton', 'Stirling', 'Richmond', 'Sydney', 'Brandon', 'Windsor', 'Scarborough', 'Palmerston'}
Australia	England	{'Stanley', 'Margate', 'Guildford', 'Warwick', 'Darlington', 'Kingston', 'York', 'Richmond', 'Ipswich', 'Sandy', 'Launceston', 'Gloucester', 'Cheltenham', 'Ascot', 'Newcastle', 'Brighton', 'Brunswick', 'Carlisle', 'Scarborough', 'Salis

In [35]:
filename = dst + 'all_institution_locations.csv'
df.to_csv(filename, index=False)

```
['Casilla C, Chile', 'Yanji, China', 'Rongcheng, China', 'Baqaa, Jordan', 'Til, Palestinian Territory', 'University Hts, AL, United States', 'Kirkland, AL, United States']
```

In [49]:
src = '../results/'
filename = src + 'all_institution_locations.csv'
df = pd.read_csv(filename).fillna('')
df['latitude'] = 0.0
df['longitude'] = 0.0

In [16]:
countries = np.sort(pd.unique(df.country))

In [ ]:
df.loc[(df.country=='Afghanistan') & (df.location=='Kabul, Afghanistan'), 'latitude'] = 98.45

In [36]:
idx = 8
countryname = countries[idx]
countryname = 'China'
dfcountry = df[df['country'] == countryname]

In [37]:
dfcountry.head(8)

,original_id,full_name,city,state,country,zip_code,location
92,33,"Sichuan Univ, Coll Life Sci, Minist Educ, Key ...",Chengdu,,China,610064,"Chengdu, China"
274,101,"Fudan Univ, Dept Environm Sci & Engn, Shanghai...",Shanghai,Shaghai,China,200433,"Shanghai, _Shaghai_, China"
439,167,"Ocean Univ China, Mariculture Lab, Coll Fisher...",Qingdao,,China,266003,"Qingdao, China"
440,167,"Ocean Univ China, Inst Marine Drug & Food, Qin...",Qingdao,,China,266003,"Qingdao, China"
568,215,"Shantou Univ, Inst Marine Biol, Shantou 515063...",Shantou,Guangdong,China,515063,"Shantou, _Guangdong_, China"
569,215,"Xiamen Univ, State Key Lab Marine Environm Sci...",Xiamen,,China,361005,"Xiamen, China"
570,215,"Chinese Acad Sci, Inst Hydrobiol, State Key La...",Wuhan,Hubei,China,430072,"Wuhan, _Hubei_, China"
571,216,"Chinese Acad Sci, Inst Hydrobiol, Wuhan 430072...",Wuhan,Hubei,China,430072,"Wuhan, _Hubei_, China"


In [38]:
uq_address = pd.unique(dfcountry['location'])
addr_dict = dict(zip(uq_address, np.zeros(len(uq_address), dtype=int)))
for key in dfcountry['location']:
    addr_dict[key] += 1
name_dict = {k: v for k, v in sorted(addr_dict.items(), key=lambda item: item[1], reverse=True)}
i = 0
if len(name_dict) > 15:
    with open('../results/locations.txt', 'w') as f:
        for key in name_dict:
            f.write('{:04d}\t{:04d}\t{}\t{}\n'.format(i,-(len(name_dict)-i), key, name_dict[key]))
            i += 1
else:
    for key in name_dict:
        print(i, -(len(name_dict)-i), key, name_dict[key], sep='\t')
        i += 1
                    
uqlocations = list(name_dict.keys())

In [40]:
CityCountry = []
CityStateCountry = []
CityCityCountry = []
ct3, ct2 = 0, 0

for i in range(len(uqlocations)):
    loc = uqlocations[i]
    foo = loc.split(', ')
    if len(foo) == 3:
        city, state, country = foo
        foo[-1] = 'wCountry'
        loc = ', '.join(foo)
        CityStateCountry.append([i, ct3, loc, -1, ''])
        locc = ', '.join([foo[0], foo[2]])
        CityCityCountry.append([i, ct3, locc, ct3, loc])
        ct3 += 1
    else:
        city, country = foo
        foo[-1] = 'wCountry'
        loc = ', '.join(foo)
        CityCountry.append([i, ct2, loc, -1, ''])
        ct2 += 1

dfCSC = pd.DataFrame(CityStateCountry, columns=['uqidx', 'idx', 'loc', 'idxloc', 'uqloc'])
print(len(CityStateCountry), 1.25*len(CityCountry))
print('City, State, Country:\t',len(CityStateCountry),'\nCity,        Country:\t', len(CityCountry), sep='')
citystatedominant = len(CityStateCountry) > 1.25*len(CityCountry)
print('citystatedominant', citystatedominant)

872 833.75
City, State, Country:	872
City,        Country:	667
citystatedominant True


In [26]:
CityCountry

[[1, 0, 'Beijing, wCountry', -1, ''],
 [3, 1, 'Wuhan, wCountry', -1, ''],
 [6, 2, 'Nanjing, wCountry', -1, ''],
 [8, 3, 'Shanghai, wCountry', -1, ''],
 [12, 4, 'Guangzhou, wCountry', -1, ''],
 [15, 5, 'Chengdu, wCountry', -1, ''],
 [16, 6, 'Hangzhou, wCountry', -1, ''],
 [17, 7, 'Lanzhou, wCountry', -1, ''],
 [18, 8, 'Kunming, wCountry', -1, ''],
 [21, 9, 'Harbin, wCountry', -1, ''],
 [22, 10, 'Qingdao, wCountry', -1, ''],
 [23, 11, 'Shenyang, wCountry', -1, ''],
 [24, 12, 'Fuzhou, wCountry', -1, ''],
 [25, 13, 'Zhengzhou, wCountry', -1, ''],
 [27, 14, 'Jinan, wCountry', -1, ''],
 [28, 15, 'Chongqing, wCountry', -1, ''],
 [30, 16, 'Changchun, wCountry', -1, ''],
 [31, 17, 'Tianjin, wCountry', -1, ''],
 [34, 18, 'Shenzhen, wCountry', -1, ''],
 [41, 19, 'Hefei, wCountry', -1, ''],
 [43, 20, 'Urumqi, wCountry', -1, ''],
 [44, 21, 'Nanning, wCountry', -1, ''],
 [45, 22, 'Xian, wCountry', -1, ''],
 [47, 23, 'Yangling, wCountry', -1, ''],
 [50, 24, 'Changsha, wCountry', -1, ''],
 [52, 25, 'B

845 880.0


In [ ]:
reload(plb)
idx = 904
loc = CityStateCountry[idx][2]
print(loc)

plb.fuzzymatch_usa(loc, CityStateCountry[:idx])

In [ ]:
reload(plb)
if countryname == 'USA':
    idx = 0
    CityStateCountry[idx][3] = CityStateCountry[idx][1]
    CityStateCountry[idx][4] = CityStateCountry[idx][2]
            
    for idx in range(1, len(CityStateCountry)):
        loc = CityStateCountry[idx][2]
        jdx, mm, score = plb.fuzzymatch_usa(loc, CityStateCountry[:idx])
        if score > 10:
            CityStateCountry[idx][3] = jdx
            CityStateCountry[idx][4] = mm
        else:
            CityStateCountry[idx][3] = CityStateCountry[idx][1]
            CityStateCountry[idx][4] = loc

In [ ]:
thr = 94
idx = 0
loc = CityStateCountry[idx][2]
citycountry, score = plb.city2citystate(loc, CityCountry, thr=95)
print(loc, '-->', citycountry, score, sep='\t')
if score < thr:
    city, state, country = loc.split(', ')
    loc = ', '.join([state[1:-1].replace('_',' '), '_'+city.replace(' ','_')+'_', country])
    citycountry, score = plb.city2citystate(loc, CityCountry, thr=95)
    print(loc, '-->', citycountry, score, sep='\t')
if score < thr:
    loc = ', '.join([city, country])
    citycountry, score = plb.city2citystate(loc, CityCountry, thr=95)
    print(loc, '-->', citycountry, score, sep='\t')
if score < thr:
    loc = ', '.join([state[1:-1].replace('_',' '), country])
    citycountry, score = plb.city2citystate(loc, CityCountry, thr=95)
    print(loc, '-->', citycountry, score, sep='\t')
    
plb.city2citystate(loc, CityCountry, thr=95)

In [ ]:
thr = 95
if not citystatedominant:
    for idx in range(len(CityStateCountry)):
        loc = CityStateCountry[idx][2]
        citycountry, score = plb.city2citystate(loc, CityCountry, thr=thr)
        
        if score < thr:
            city, state, country = loc.split(', ')
            locc = ', '.join([state[1:-1].replace('_',' '), '_'+city.replace(' ','_')+'_', country])
            citycountry, score = plb.city2citystate(locc, CityCountry, thr=thr)
        if score < thr:
            locc = ', '.join([city, country])
            citycountry, score = plb.city2citystate(locc, CityCountry, thr=thr)
        if score < thr:
            locc = ', '.join([state[1:-1].replace('_',' '), country])
            citycountry, score = plb.city2citystate(locc, CityCountry, thr=thr)
            
        if score < thr:
            CityStateCountry[idx][3] = -1
            CityStateCountry[idx][4] = loc
        else:
            CityStateCountry[idx][3] = citycountry[1]
            CityStateCountry[idx][4] = citycountry[2]        

In [ ]:
for idx in range(len(CityStateCountry)):
    print(idx, CityStateCountry[idx], sep='\t')

In [ ]:
reload(plb)
idx = 0
loc = CityCountry[idx][2]
print(loc)
plb.city2citystate(loc, CityCountry[:idx], thr=94)

In [ ]:
foo = 'Hawali, wCountry'
bar = 'Hawally, wCountry'
plb.fuzzscore(foo, bar)

In [ ]:
thr = 94
if not citystatedominant:
    idx = 0
    CityCountry[idx][3] = 0
    CityCountry[idx][4] = CityCountry[idx][2]
    for idx in range(1, len(CityCountry)):
        loc = CityCountry[idx][2]
        citycountry, score = plb.city2citystate(loc, CityCountry[:idx], thr)
        if score < thr:
            CityCountry[idx][3] = idx
            CityCountry[idx][4] = loc
        else:
            CityCountry[idx][3] = citycountry[1]
            CityCountry[idx][4] = citycountry[2]     

In [ ]:
for idx in range(len(CityCountry)):
    print(idx, CityCountry[idx], sep='\t')

In [ ]:
reload(plb)

thr = 89

i = 1
loc = CityCountry[i][2]
#citycountry, score = city2city(CityCountry[i], CityCountry[:i], thr=thr)
#print(CityCountry[i][1], '-->', citycountry[1], score, sep='\t')

citystatecountry, score = plb.city2citystate(loc, CityCityCountry, thr=90)
print(loc, '-->', uqlocations[citystatecountry[0]], score, sep='\t')

citystatecountry, score = plb.city2citystate(loc, CityStateCountry, thr=89)
print(loc, '-->', uqlocations[citystatecountry[0]], score, sep='\t')

locmode = plb.city2zipcode(loc, dfcountry, dfCSC)
print(loc, '-->', locmode, sep='\t')

In [ ]:
zipcodes = dfcountry[dfcountry['location'] == loc]['zip_code']
zipcode = zipcodes[zipcodes != ''].mode().values[0]
dfcountry[dfcountry['zip_code'] == zipcode].head()#['location'].mode().values[0]

In [ ]:
thr = 89
if citystatedominant and countryname != 'USA':
    for i in range(len(CityCountry)):
        loc = CityCountry[i][2]
        citystatecountry, score = plb.city2citystate(loc, CityCityCountry, thr=92)
        if score > thr:
            CityCountry[i][-2] = citystatecountry[1]
            CityCountry[i][-1] = uqlocations[citystatecountry[0]]
        else:
            citystatecountry, score = plb.city2citystate(loc, CityStateCountry, thr=89)
            if score > thr:
                CityCountry[i][-2] = citystatecountry[1]
                CityCountry[i][-1] = uqlocations[citystatecountry[0]]
            else:
                locmode = plb.city2zipcode(loc, dfcountry, dfCSC)
                CityCountry[i][-2] = locmode[1]
                CityCountry[i][-1] = locmode[0]

        print(i, CityCountry[i], sep='\t')

In [ ]:
reload(plb)
verbose = True
i = 103
loc = CityStateCountry[i][2]
city, state, country = loc.split(', ')

print('====', loc,'\n')
jdx, match, score = plb.fuzzymatch_up(loc, CityCityCountry[:i], thr=90, verbose=verbose)
if jdx == -1:
    invloc = ', '.join([state[1:-1], '_' + city + '_', country])
    print('\n====',invloc)
    jdx, match, score = plb.fuzzymatch_up(invloc, CityCityCountry[:i], thr=90, fuzzscore_val=[0], matchstate=True, verbose=verbose)

if jdx == -1:
    jdx = i
    match = loc
    score = 0    
    
print('\n-------\n')
print(jdx, match, score, sep='\t')

In [ ]:
verbose = False
if citystatedominant and countryname != 'USA':
    idx = 0
    CityStateCountry[idx][-1] = CityStateCountry[idx][2]
    CityStateCountry[idx][-2] = idx
    print(idx, CityStateCountry[idx], sep='\t')

    for idx in range(1, len(CityStateCountry)):
        loc = CityStateCountry[idx][2]
        city, state, country = loc.split(', ')

        jdx, match, score = plb.fuzzymatch_up(loc, CityCityCountry[:idx], thr=90, verbose=verbose)
        if jdx == -1:
            invloc = ', '.join([state[1:-1], '_' + city + '_', country])
            jdx, match, score = plb.fuzzymatch_up(invloc, CityCityCountry[:idx], thr=90, fuzzscore_val=[0], matchstate=True, verbose=verbose)

        if jdx == -1:
            jdx = idx
            match = loc
            score = 0    

        CityStateCountry[idx][-1] = match
        CityStateCountry[idx][-2] = jdx

        print(idx, CityStateCountry[idx], sep='\t')

In [ ]:
if citystatedominant and countryname != 'USA':
    CityStateCountry = plb.upflow_update(CityCityCountry)
else:
    CityCountry = plb.upflow_update(CityCountry)

In [ ]:
for idx in range(len(CityCountry)):
    for jdx in [2,4]:
        foo = CityCountry[idx][jdx].split(', ')
        CityCountry[idx][jdx] = ', '.join(foo[:-1] + [countryname])
for idx in range(len(CityStateCountry)):
    for jdx in [2,4]:
        foo = CityStateCountry[idx][jdx].split(', ')
        CityStateCountry[idx][jdx] = ', '.join(foo[:-1] + [countryname])

In [ ]:
if citystatedominant:
    locs = dfcountry['location'].values.copy()
    for idx in range(1,len(CityStateCountry)):
        i, jdx, loc, kdx, match = CityStateCountry[idx]
        if jdx != kdx:
            locs[locs == loc] = match

    for idx in range(len(CityCountry)):
        i, jdx, loc, kdx, match = CityCountry[idx]
        if kdx != -1:
            i, jdx, _, kdx, match = CityStateCountry[kdx]
            locs[locs == loc] = match

In [ ]:
if not citystatedominant:
    locs = dfcountry['location'].values.copy()
    for idx in range(1,len(CityCountry)):
        i, jdx, loc, kdx, match = CityCountry[idx]
        if jdx != kdx:
            locs[locs == loc] = match

    for idx in range(len(CityStateCountry)):
        i, jdx, loc, kdx, match = CityStateCountry[idx]
        if kdx != -1:
            i, jdx, _, kdx, match = CityCountry[kdx]
            locs[locs == loc] = match

In [ ]:
uq_locs = pd.unique(locs)
locs_dict = dict(zip(uq_locs, np.zeros(len(uq_locs), dtype=int)))
for key in locs:
    locs_dict[key] += 1
name_dict = {k: v for k, v in sorted(locs_dict.items(), key=lambda item: item[1], reverse=True)}

i = 1
if len(name_dict) > 15:
    filename = '../results/uqlocations_'+countryname.replace(' ', '-').lower()+'.tsv'
    with open(filename, 'w') as f:
        for key in name_dict:
            f.write('{:04d}\t{:04d}\t{}\t{}\n'.format(i,-(len(name_dict)-i+1), key, name_dict[key]))
            i += 1
else:
    for key in name_dict:
        print(i, -(len(name_dict)-i+1), key, name_dict[key], sep='\t')
        i += 1

In [ ]:
np.nonzero(locs == 'Middelburg, Eastern Cape, South Africa')

In [ ]:
for idx in range(len(locs)):
    loc = locs[idx]
    foo = loc.split(', ')
    if len(foo) == 3:
        city, state, country = foo
        loc = ', '.join([city, state[1:-1].replace('_',' '), countryname])
        locs[idx] = loc

In [ ]:
df.loc[df['country']==country, 'location'] = locs

In [ ]:
for i in range(len(countries)):
    print(i, countries[i], sep='\t')

In [ ]:
country_dict = dict(zip(countries, np.zeros(len(countries), dtype=int)))
for key in df['country']:
    country_dict[key] += 1
name_dict = {k: v for k, v in sorted(country_dict.items(), key=lambda item: item[1], reverse=True)}
i = 1
for key in name_dict:
    print(i, key, name_dict[key], sep='\t')
    i += 1

In [ ]:
filename = '../results/split_locations_of_all_authors_20220423.csv'
df.to_csv(filename, index=False)

# Once all geolocations have been computed

In [28]:
print(data.shape)

(296447, 53)


In [27]:
filename = '../results/author_coauthor_corresponding.csv'
metainfo = pd.read_csv(filename)
print(metainfo.shape)
metainfo.head()

(296447, 3)


,coauthor_num,corresponding_num,corr_institutions
0,3,1,0
1,2,1,0
2,4,1,0
3,6,1,0
4,3,1,0


In [101]:
paper = df[df['original_id']  == 62140]
paper

,original_id,full_name,city,state,country,zip_code,location,latitude,longitude
150791,62140,"IASBS, Dept Chem, Zanjan 4513766731, Iran",Zanjan,,Iran,4513766731,"Zanjan, Iran",36.683005,48.508721
150792,62140,"IASBS, Ctr Climate Change & Global Warming, Za...",Zanjan,,Iran,4513766731,"Zanjan, Iran",36.683005,48.508721
150793,62140,"IASBS, Res Ctr Basic Sci & Modern Technol RBST...",Zanjan,,Iran,4513766731,"Zanjan, Iran",36.683005,48.508721
150794,62140,"IASBS, Dept Biol Sci, Zanjan 451951159, Iran",Zanjan,,Iran,451951159,"Zanjan, Iran",36.683005,48.508721
150795,62140,"Chinese Acad Sci, Ningbo Inst Mat Technol & En...",Ningbo,Zhejiang,China,315201,"Ningbo, Zhejiang, China",29.868336,121.543990
150796,62140,"Russian Acad Sci, Inst Plant Physiol, Controll...",Moscow,,Russia,127276,"Moscow, Russia",55.755826,37.617300
150797,62140,"Russian Acad Sci, Inst Basic Biol Problems, In...",Pushchino,Moscow Region,Russia,142290,"Pushchino, Russia",54.832386,37.621056
150798,62140,"Moscow MV Lomonosov State Univ, Dept Plant Phy...",Moscow,,Russia,119991,"Moscow, Russia",55.755826,37.617300
150799,62140,"Moscow Inst Phys & Technol, Inst Sky Lane 9, D...",Dolgoprudnyi,Moscow Region,Russia,141700,"Dolgoprudnyi, Russia",55.947064,37.499276
150800,62140,"Azerbaijan Natl Acad Sci, Inst Mol Biol & Biot...",Baku,,Azerbaijan,1073,"Baku, Azerbaijan",40.409262,49.867092


In [102]:
paper.iloc[np.asarray(metainfo.loc[62140, 'corr_institutions'].split(','), dtype=int),:]

,original_id,full_name,city,state,country,zip_code,location,latitude,longitude
150791,62140,"IASBS, Dept Chem, Zanjan 4513766731, Iran",Zanjan,,Iran,4513766731,"Zanjan, Iran",36.683005,48.508721
150792,62140,"IASBS, Ctr Climate Change & Global Warming, Za...",Zanjan,,Iran,4513766731,"Zanjan, Iran",36.683005,48.508721
150793,62140,"IASBS, Res Ctr Basic Sci & Modern Technol RBST...",Zanjan,,Iran,4513766731,"Zanjan, Iran",36.683005,48.508721
150796,62140,"Russian Acad Sci, Inst Plant Physiol, Controll...",Moscow,,Russia,127276,"Moscow, Russia",55.755826,37.617300
150797,62140,"Russian Acad Sci, Inst Basic Biol Problems, In...",Pushchino,Moscow Region,Russia,142290,"Pushchino, Russia",54.832386,37.621056
150798,62140,"Moscow MV Lomonosov State Univ, Dept Plant Phy...",Moscow,,Russia,119991,"Moscow, Russia",55.755826,37.617300
150799,62140,"Moscow Inst Phys & Technol, Inst Sky Lane 9, D...",Dolgoprudnyi,Moscow Region,Russia,141700,"Dolgoprudnyi, Russia",55.947064,37.499276
150800,62140,"Azerbaijan Natl Acad Sci, Inst Mol Biol & Biot...",Baku,,Azerbaijan,1073,"Baku, Azerbaijan",40.409262,49.867092


In [23]:
filename = '../results/split_locations_of_all_authors_20220505.csv'
df = pd.read_csv(filename).fillna('')
dflocs = df['location']
print(df.shape)
df.head()

(786897, 9)


,original_id,full_name,city,state,country,zip_code,location,latitude,longitude
0,0,"Univ Autonoma Madrid, Dept Biol Mol, E-28049 M...",Madrid,,Spain,E-28049,"Madrid, Mostoles, Spain",40.323213,-3.867629
1,0,"IEO, Ctr Oceanog Vigo, Vigo 36390, Spain",Vigo,,Spain,36390,"Vigo, Pontevedra, Spain",42.240599,-8.720727
2,0,"MIT, Dept Civil & Environm Engn, 77 Massachuse...",Cambridge,MA,United States,02139,"Cambridge, MA, United States",42.373616,-71.109733
3,1,"Assiut Univ, Fac Sci, Dept Bot & Microbiol, As...",Assiut,,Egypt,71516,"Assiut, Egypt",27.178312,31.185926
4,1,"Assiut Univ, Fac Agr, Dept Plant Pathol, Assiu...",Assiut,,Egypt,71526,"Assiut, Egypt",27.178312,31.185926


In [120]:
countrys = ['' for i in range(len(data))]
locs = ['' for i in range(len(data))]
latlon = ['' for i in range(len(data))]
country_num = [0 for i in range(len(data))]

ccountrys = ['' for i in range(len(data))]
clocs = ['' for i in range(len(data))]
clatlon = ['' for i in range(len(data))]
ccountry_num = [0 for i in range(len(data))]

In [126]:
ctr = 0
i = 0
for i in range(len(data)):
    authnum = metainfo.loc[i, 'coauthor_num']
    paper = df.iloc[ctr:ctr+authnum, :]
    
    authcountry = pd.unique(paper['country'])
    countrys[i] = '; '.join(authcountry)
    locs[i] = '; '.join(pd.unique(paper['location']))

    foo = str(list(zip(paper['latitude'], paper['longitude'])))
    bar = [x + ')' for x in foo[1:-1].split('), ')]
    bar[-1] = bar[-1][:-1]
    latlon[i] = '; '.join(np.unique(bar))
    
    country_num[i] = len(authcountry)
    
    corrnum = np.asarray(metainfo.loc[i, 'corr_institutions'].split(','), dtype=int)
    cpaper = paper.iloc[corrnum,:]
    
    corrcountry = pd.unique(cpaper['country'])
    ccountrys[i] = '; '.join(corrcountry)
    clocs[i] = '; '.join(pd.unique(cpaper['location']))

    foo = str(list(zip(cpaper['latitude'], cpaper['longitude'])))
    bar = [x + ')' for x in foo[1:-1].split('), ')]
    bar[-1] = bar[-1][:-1]
    clatlon[i] = '; '.join(np.unique(bar))
    
    ccountry_num[i] = len(corrcountry)

    ctr += authnum

In [127]:
paper

,original_id,full_name,city,state,country,zip_code,location,latitude,longitude
786895,296446,"Ctr Univ Djelfa, Inst Sci Nat & Vie, Djelfa 17...",Djelfa,,Algeria,17000,"Djelfa, Algeria",34.670396,3.250376
786896,296446,"Inst Natl Agr, Dept Sci Sol El Harrach, Algier...",Algiers,,Algeria,,"Algiers, Algeria",36.753768,3.058756


In [128]:
for i in range(5):
    print(countrys[i], country_num[i], locs[i], latlon[i], 
          '----', 
          ccountrys[i], ccountry_num[i], clocs[i], clatlon[i], sep='\n')
    print('~|~|~|~')

Spain; United States
2
Madrid, Mostoles, Spain; Vigo, Pontevedra, Spain; Cambridge, MA, United States
(40.3232129, -3.8676291); (42.24059889999999, -8.720726800000001); (42.3736158, -71.10973349999998)
----
Spain
1
Madrid, Mostoles, Spain
(40.3232129, -3.8676291)
~|~|~|~
Egypt
1
Assiut, Egypt
(27.1783117, 31.1859257)
----
Egypt
1
Assiut, Egypt
(27.1783117, 31.1859257)
~|~|~|~
Italy; United States
2
Rome, RM, Italy; Sez Istol, Parma, Italy; Athens, OH, United States
(39.3292396, -82.1012554); (41.90278350000001, 12.4963655); (44.801485, 10.3279036)
----
Italy
1
Rome, RM, Italy
(41.90278350000001, 12.4963655)
~|~|~|~
Spain; Australia; England; Brazil
4
Murcia, Espinardo, Spain; Armilla, Granada, Spain; Parkville, Vic, Australia; Darlington, Durham, England; Sao Jose Do Rio Preto, SP, Brazil
(-20.8127115, -49.376521); (-37.7787036, 144.94260830000005); (37.15123060000001, -3.6167146); (38.0111896, -1.1556962); (54.52361, -1.559458)
----
Spain
1
Murcia, Espinardo, Spain
(38.0111896, -1.155

In [129]:
data['Nations--all authors'] = countrys
data['Corresponding author Nation'] = ccountrys

In [132]:
data['Locations--all authors'] = locs
data['Geocoords--all authors'] = latlon

data['Corresponding author locations'] = clocs
data['Corresponding author geocoords'] = clatlon

data['Number of authoring countries'] = country_num
data['Number of corresponding countries'] = ccountry_num

In [134]:
filename = '../data/Plt_sci_publications_geocoord_8.18.csv'
print(data.shape)
data.to_csv(filename, index=False)

(296447, 59)
